# Introdução

**Integrantes:**
- Francisco Lou - 2211275
- Vinicius Araujo - 2210392
- Felipe Fortini - 2220501
- Luca Lima - 2210831
- João Felipe Motta - 2211721

Neste trabalho, iremos:

- **Apresentar o objetivo**: Trabalhar com os dados e usar modelos para classificar risco de crédito.
- **Descrever o dataset**: "German Credit", composto por 1.000 registros e 9 atributos selecionados.


In [2]:
# imports
import pandas as pd


In [3]:
#Carregar dados
df = pd.read_csv('class_german_credit.csv')

# Mostrar dimensões 
print("Dimensões:", df.shape)

df.info()

# Mostrar primeiras linhas de dados
df.head()

Dimensões: (1000, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               1000 non-null   int64 
 1   Sex               1000 non-null   object
 2   Job               1000 non-null   int64 
 3   Housing           1000 non-null   object
 4   Saving accounts   817 non-null    object
 5   Checking account  606 non-null    object
 6   Credit amount     1000 non-null   int64 
 7   Duration          1000 non-null   int64 
 8   Purpose           1000 non-null   object
 9   Risk              1000 non-null   object
dtypes: int64(4), object(6)
memory usage: 78.3+ KB


,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


# Descrição dos Atributos

### 1. **Age**
- **Tipo:** Numérico
- **Escala:** Razão
- **Cardinalidade:** Contínua
- **Explicação:** Idade em anos, com zero significativo e intervalo praticamente contínuo.

### 2. **Sex**
- **Tipo:** Categórico
- **Escala:** Nominal
- **Cardinalidade:** Binária
- **Explicação:** Male/female, sem hierarquia e com duas categorias.

### 3. **Job**
- **Tipo:** Numérico
- **Escala:** Razão
- **Cardinalidade:** Discreta
- **Explicação:** Níveis de emprego codificados em inteiros de 0 a 3.

### 4. **Housing**
- **Tipo:** Categórico
- **Escala:** Nominal
- **Cardinalidade:** Discreta
- **Explicação:** Tipo de moradia (own/rent/free), sem ordem.

### 5. **Saving accounts**
- **Tipo:** Categórico
- **Escala:** Nominal
- **Cardinalidade:** Discreta
- **Explicação:** Categorias de saldo (little, moderate, quite rich, rich).

### 6. **Checking account**
- **Tipo:** Categórico
- **Escala:** Nominal
- **Cardinalidade:** Discreta
- **Explicação:** Categorias de saldo em conta corrente (little, moderate, rich, etc.), sem relação ordinal numérica.

### 7. **Credit amount**
- **Tipo:** Numérico
- **Escala:** Razão
- **Cardinalidade:** Contínua
- **Explicação:** Valor do empréstimo em DM, com zero significativo e contínuo.

### 8. **Duration**
- **Tipo:** Numérico
- **Escala:** Razão
- **Cardinalidade:** Discreta
- **Explicação:** Duração em meses, valores inteiros.

### 9. **Purpose**
- **Tipo:** Categórico
- **Escala:** Nominal
- **Cardinalidade:** Discreta
- **Explicação:** Finalidade do crédito, sem hierarquia.

### 10. **Risk**
- **Tipo:** Categórico
- **Escala:** Nominal
- **Cardinalidade:** Binária
- **Explicação:** Alvo do modelo, good/bad.

In [4]:
descr = pd.DataFrame({
    'Atributo': [
        'Age', 'Sex', 'Job', 'Housing',
        'Saving accounts', 'Checking account',
        'Credit amount', 'Duration', 'Purpose', 'Risk'
    ],
    'Tipo': [
        'Numérico', 'Categórico', 'Numérico', 'Categórico',
        'Categórico', 'Categórico',
        'Numérico', 'Numérico', 'Categórico', 'Categórico'
    ],
    'Escala': [
        'Razão', 'Nominal', 'Razão', 'Nominal',
        'Nominal', 'Nominal',
        'Razão', 'Razão', 'Nominal', 'Nominal'
    ],
    'Cardinalidade': [
        'Contínua', 'Binária', 'Discreta', 'Discreta',
        'Discreta', 'Discreta',
        'Contínua', 'Discreta', 'Discreta', 'Binária'
    ]
})

descr

,Atributo,Tipo,Escala,Cardinalidade
0,Age,Numérico,Razão,Contínua
1,Sex,Categórico,Nominal,Binária
2,Job,Numérico,Razão,Discreta
3,Housing,Categórico,Nominal,Discreta
4,Saving accounts,Categórico,Nominal,Discreta
5,Checking account,Categórico,Nominal,Discreta
6,Credit amount,Numérico,Razão,Contínua
7,Duration,Numérico,Razão,Discreta
8,Purpose,Categórico,Nominal,Discreta
9,Risk,Categórico,Nominal,Binária
